In [1]:
import numpy as np
import pandas as pd
import json

In [2]:
!conda install beautifulsoup4
!conda install lxml
!conda install html5lib
!conda install requests

Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    beautifulsoup4: 4.6.0-py35h442a8c9_1 --> 4.6.3-py35_0

beautifulsoup4 100% |################################| Time: 0:00:00  27.30 MB/s
Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following packages will be UPDATED:

    libgcc-ng: 7.2.0-h7cc24e2_2     --> 8.2.0-hdf63c60_1    
    libxml2:   2.9.4-h6b072ca_5     --> 2.9.8-hf84eae3_0    
    libxslt:   1.1.29-hcf9102b_5    --> 1.1.33-h7d1a2b0_0   
    lxml:      4.1.0-py35ha401a81_0 --> 4.2.5-py35hefd8a0e_0

libgcc-ng-8.2. 100% |################################| Time: 0:00:00  73.13 MB/s
libxml2-2.9.8- 100% |################################| Time: 0:00:00  24.08 MB/s
libxslt-1.1.33 100% |################################| Time: 0:00:00  32.

In [3]:
from bs4 import BeautifulSoup
import requests
wikiweb_link="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source=requests.get(wikiweb_link).text

In [4]:
soup=BeautifulSoup(source,'lxml')

In [5]:
citytable=soup.find('table',class_="wikitable sortable")
dfcity = pd.read_html(str(citytable), header=0)[0]

In [6]:
dfcity['Postcode'] = dfcity['Postcode'].astype(str)
dfcity['Borough'] = dfcity['Borough'].astype(str)
dfcity['Neighbourhood'] = dfcity['Neighbourhood'].astype(str)

In [8]:
dfcity.head()

Postcode           Borough     Neighbourhood
0      M1A      Not assigned      Not assigned
1      M2A      Not assigned      Not assigned
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront

In [9]:
dfcity=dfcity.drop(dfcity[dfcity.Borough=='Not assigned'].index)
dfcity.head()

Postcode           Borough     Neighbourhood
2      M3A        North York         Parkwoods
3      M4A        North York  Victoria Village
4      M5A  Downtown Toronto      Harbourfront
5      M5A  Downtown Toronto       Regent Park
6      M6A        North York  Lawrence Heights

In [10]:
dfcity=dfcity.groupby(['Postcode',"Borough"])["Neighbourhood"].apply(lambda x: ','.join(x)).reset_index()
dfcity.head()

Postcode      Borough                         Neighbourhood
0      M1B  Scarborough                         Rouge,Malvern
1      M1C  Scarborough  Highland Creek,Rouge Hill,Port Union
2      M1E  Scarborough       Guildwood,Morningside,West Hill
3      M1G  Scarborough                                Woburn
4      M1H  Scarborough                             Cedarbrae

In [11]:
dfcity['Neighbourhood'].loc[dfcity['Neighbourhood'].str.contains('Not assigned')]=dfcity['Borough']

wiwi=dfcity[dfcity['Postcode'].str.contains('M7A')]
wiwi

Postcode       Borough Neighbourhood
85      M7A  Queen's Park  Queen's Park

In [12]:
dfcity.shape

(103, 3)

In [13]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.19.0-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  22.18 MB/s
geopy-1.19.0-p 100% |################################| Time: 0:00:00  32.66 MB/s


In [15]:
column_names=['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']
neighborhoods = pd.DataFrame(columns=column_names)

In [17]:
for data in dfcity.itertuples():
    address = ' {0}, {1}, Ontario'.format(data[1],data[2])
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    
    try:
        latitude = location.latitude
        longitude = location.longitude
    except: 
        latitude =0
        longitude = 0

        
    neighborhoods = neighborhoods.append({'Postcode': data[1],
                                        'Borough': data[2],
                                        'Neighborhood': data[3],
                                        'Latitude': latitude,
                                        'Longitude': longitude}, ignore_index=True)
    
neighborhoods.head()

Postcode      Borough                          Neighborhood   Latitude  \
0      M1B  Scarborough                         Rouge,Malvern  43.773077   
1      M1C  Scarborough  Highland Creek,Rouge Hill,Port Union  43.773077   
2      M1E  Scarborough       Guildwood,Morningside,West Hill   0.000000   
3      M1G  Scarborough                                Woburn   0.000000   
4      M1H  Scarborough                             Cedarbrae   0.000000   

   Longitude  
0 -79.257774  
1 -79.257774  
2   0.000000  
3   0.000000  
4   0.000000

In [18]:
neighborhoods['Latitude'].value_counts()

0.000000     170
42.858062      6
43.773077      4
43.756199      2
43.662173      2
43.816435      2
43.649248      2
43.644903      2
43.654374      2
43.659980      2
43.675566      2
43.732822      2
43.763531      2
43.718905      1
Name: Latitude, dtype: int64

In [19]:
!wget -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv


--2019-04-16 13:07:19--  http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv [following]
--2019-04-16 13:07:19--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-04-16 13:07:20--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.25.197, 107.152.24.197
Connecting to ibm.box.com (ibm.box.com)|107.152.25.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18r

In [20]:
coorsourcedp = pd.read_csv("Geospatial_Coordinates.csv")
coorsourcedp.head()

Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476

In [21]:
dummydf=coorsourcedp[coorsourcedp['Postal Code']=='M1C']   
tlat=dummydf['Latitude'].values[0]
tlong=dummydf['Longitude'].values[0]
print(tlat)

43.7845351


In [22]:
column_names=['Postcode', 'Borough', 'Neighborhood', 'Latitude', 'Longitude']

neighborhoods = pd.DataFrame(columns=column_names)

neighborhoods['Latitude'] = neighborhoods['Latitude'].astype(np.float64)
neighborhoods['Longitude'] = neighborhoods['Longitude'].astype(np.float64)

tlat=0.0
tlong=0.0

for data in dfcity.itertuples():
    dummydf=coorsourcedp[coorsourcedp['Postal Code']==data[1]]   
    tlat=dummydf['Latitude'].values[0]
    tlong=dummydf['Longitude'].values[0]
    
    neighborhoods = neighborhoods.append({'Postcode': data[1],
                                        'Borough': data[2],
                                        'Neighborhood': data[3],
                                        'Latitude': tlat,
                                        'Longitude': tlong}, ignore_index=True)

print(neighborhoods.dtypes)

Postcode         object
Borough          object
Neighborhood     object
Latitude        float64
Longitude       float64
dtype: object


In [23]:
neighborhoods.head(15)

Postcode      Borough                                       Neighborhood  \
0       M1B  Scarborough                                      Rouge,Malvern   
1       M1C  Scarborough               Highland Creek,Rouge Hill,Port Union   
2       M1E  Scarborough                    Guildwood,Morningside,West Hill   
3       M1G  Scarborough                                             Woburn   
4       M1H  Scarborough                                          Cedarbrae   
5       M1J  Scarborough                                Scarborough Village   
6       M1K  Scarborough          East Birchmount Park,Ionview,Kennedy Park   
7       M1L  Scarborough                      Clairlea,Golden Mile,Oakridge   
8       M1M  Scarborough      Cliffcrest,Cliffside,Scarborough Village West   
9       M1N  Scarborough                         Birch Cliff,Cliffside West   
10      M1P  Scarborough  Dorset Park,Scarborough Town Centre,Wexford He...   
11      M1R  Scarborough                                   Maryvale,Wexford   
12      M1S  Scarborough                                          Agincourt   
13      M1T  Scarborough              Clarks Corners,Sullivan,Tam O'Shanter   
14      M1V  Scarborough  Agincourt North,L'Amoreaux East,Milliken,Steel...   

     Latitude  Longitude  
0   43.806686 -79.194353  
1   43.784535 -79.160497  
2   43.763573 -79.188711  
3   43.770992 -79.216917  
4   43.773136 -79.239476  
5   43.744734 -79.239476  
6   43.727929 -79.262029  
7   43.711112 -79.284577  
8   43.716316 -79.239476  
9   43.692657 -79.264848  
10  43.757410 -79.273304  
11  43.750072 -79.295849  
12  43.794200 -79.262029  
13  43.781638 -79.304302  
14  43.815252 -79.284577